In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px

## Import microprobe data

In [3]:
# I copied microprobe data from the Excel appendix and saved it as a .csv file,
# major_els.csv
# I omitted the Appendix's first 11 rows with references and comments

# import data as a dataframe
major_els = pd.read_csv('major_els.csv')

In [4]:
#%% save basic statistics

major_els.describe().to_csv('major_els_stats.csv')

# it looks like we have 375 data points for all the columns,
# but in reality the original dataset has more columns

# for example, the original dataset has 19 columns with major and minor elements,
# while basic statistic indicators were calculated only for 10 columns

# this is because the original dataset contains values below detection limit, <DL
# we need to check how many of those values are present, and either drop some rows 
# or replace those values with some other value that is acceptable

In [5]:
# replace "<DL" with NA, or not-a-number, and see how the statistics changes
major_els_update = major_els.replace('<DL', np.nan)
print(major_els_update.isna().sum())
major_els_update.describe().to_csv('major_els_update.csv')
# Ti, V, K are present in this list, and don't have any values below detection limit.
# and yet the describe() function did not see those columns. Why? Maybe because of spaces in the column names? 

LOCALITY        0
TS_ID           0
POINT_ID        0
ZONE            0
ANALYSE_ID      0
*FeO_pct        0
Cr2O3_pct       0
CaO_pct         0
Na2O_pct        0
MgO_pct         0
SiO2_pct        0
Al2O3_pct       0
 Mn _ ppm      94
 Ti _ ppm       0
 Zn _ ppm      14
 Cu _ ppm     305
 Ni _ ppm      98
 Co _ ppm     186
 V _ ppm        0
 Sc _ ppm      43
 K _ ppm        0
 Cl _ ppm     231
 Sr _ ppm     223
 F _ ppm      148
dtype: int64


In [7]:
# let's get rid of spaces in the column names
cols_list = list(major_els_update.columns)
cols = [x.replace(' ', '') for x in cols_list] # here we are replacing spaces with nothing
# I love list comprehension
major_els_update.columns = cols
major_els_update.describe().to_csv('major_els_update_stats.csv')
# still not there! Let's check if the columns are of different types

In [8]:
print(major_els_update.dtypes)

LOCALITY       object
TS_ID          object
POINT_ID       object
ZONE           object
ANALYSE_ID     object
*FeO_pct      float64
Cr2O3_pct     float64
CaO_pct       float64
Na2O_pct      float64
MgO_pct       float64
SiO2_pct      float64
Al2O3_pct     float64
Mn_ppm         object
Ti_ppm          int64
Zn_ppm         object
Cu_ppm         object
Ni_ppm         object
Co_ppm         object
V_ppm           int64
Sc_ppm         object
K_ppm           int64
Cl_ppm         object
Sr_ppm         object
F_ppm          object
dtype: object


In [9]:
#%% the columns that should have numeric values
els_cols = ['*FeO_pct', 'Cr2O3_pct', 'CaO_pct', 'Na2O_pct', 'MgO_pct', 'SiO2_pct',
            'Al2O3_pct', 'Mn_ppm', 'Ti_ppm', 'Zn_ppm', 'Cu_ppm', 'Ni_ppm', 'Co_ppm',
            'V_ppm', 'Sc_ppm', 'K_ppm', 'Cl_ppm', 'Sr_ppm', 'F_ppm']
major_els_numeric = major_els_update[els_cols].astype(float)
major_els_numeric.describe().to_csv('numeric_stats.csv') # and now we see stats for all the columns!

In [10]:
#%% it is not advisable to use columns with more than 30% missing values
# in our case, 30% of the dataset with 375 points equals 112,
# so the columns that we can use should have "counts" of at least 263
print(major_els_numeric.count() > 263)

*FeO_pct      True
Cr2O3_pct     True
CaO_pct       True
Na2O_pct      True
MgO_pct       True
SiO2_pct      True
Al2O3_pct     True
Mn_ppm        True
Ti_ppm        True
Zn_ppm        True
Cu_ppm       False
Ni_ppm        True
Co_ppm       False
V_ppm         True
Sc_ppm        True
K_ppm         True
Cl_ppm       False
Sr_ppm       False
F_ppm        False
dtype: bool


In [11]:
#%% 
# make a list of the numerical columns to keep
cols_to_keep = []
for col in major_els_numeric.columns:
    if major_els_numeric[col].dtype == np.number and major_els_numeric[col].count() > 263:
        cols_to_keep.append(col)


C:\Users\Dina\AppData\Local\Temp\ipykernel_39568\1721044002.py:5: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if major_els_numeric[col].dtype == np.number and major_els_numeric[col].count() > 263:


In [12]:
#%% now we have a list of columns with numeric values that we can keep, and a list of columns with
# categorical values; here is how we put all the information together for the principal component analysis

cols_key_values = ['LOCALITY', 'TS_ID', 'POINT_ID', 'ZONE', 'ANALYSE_ID']

data_major = pd.concat([major_els[cols_key_values], major_els_numeric[cols_to_keep]], axis=1)

In [15]:
#%% the last step before PCA should be replacing missing values with half of the limit of detection
# I don't really know the limit of detection, so I assume it is the minimum value in the column

for col in cols_to_keep:
    # find minimum value
    col_min_half = data_major[col].min()/2
    data_major[col].fillna(col_min_half, inplace=True)
data_major.to_csv('major_filled.csv') # check how it looks now

## Do some more pre-processing and take the log10 of element data

In [16]:
#%% now we can perform PCA and plot the first 3 principal components to see
# is there a good separation between the deposits? 
# is there a difference between the core and the rim?
# how do elements group in terms of PCA?

# take the log10 value; need to replace zeros to a small value, let's go with half of min value again

for col in cols_to_keep:
    col_min_half = data_major[col][data_major[col] >0].min()/2
    data_major[col] = data_major[col].replace(0, col_min_half)

In [17]:
# to do the PCA, we use numeric columns: those that are in the cols_to_keep list
data_for_pca = np.log10(data_major[cols_to_keep])

## PCA and plotting

In [18]:
pca_results = PCA().fit_transform(data_for_pca)
pca_data = pd.DataFrame(pca_results)
pca_cols = ['PC' + str(x + 1) for x in pca_data.columns] # give PCs their true names
pca_data.columns = pca_cols

In [19]:
# we have principal components, but those were calculated based on numeric data 
# let's get the labels back

pc_final = pd.concat([major_els[cols_key_values], major_els_numeric[cols_to_keep], pca_data], axis=1)


In [41]:
# there are 26 localities, 6 of which are from James Bay. Maybe it makes sense to group those as James Bay

# make a copy of the PC dataframe
pc_aggregated = pc_final.copy()

# replace every locality that contains 'James Bay' with just 'James Bay'
pc_aggregated.loc[pc_aggregated.LOCALITY.str.contains('James Bay'), 'LOCALITY'] = 'James Bay'

# and plot

fig = px.scatter_3d(pc_aggregated, x='PC1', y='PC2', z='PC3',
              color='LOCALITY')
fig.update_traces(marker=dict(size=3))
fig.show()

In [44]:
# the plot is a bit crowded. Here is a way to get the list of localities 
pc_aggregated['LOCALITY'].unique()

array(['Hollinger', 'Dome', 'Hoyle Pond', 'Young Davidson',
       'Canadian Malartic', 'Roberto', 'James Bay', 'Excelsior',
       'Royal Hill - Rosebel', 'Pay Caro - Rosebel', 'Salsigne',
       'Essakane', 'Navachab', 'New Consort', 'Hira Buddini', 'Uti',
       'Big Bell', 'St. Ives', 'Lincoln Hill', 'Tehery Wager', 'LaRonde'],
      dtype=object)

In [45]:
# how many different localities are there?
len(pc_aggregated['LOCALITY'].unique())

21

In [55]:
# or maybe displaying contours in PC1/PC2 space will make it more illustrative?
# split the list of 21 localities into two smaller lists, 

localities_1 = ['Hollinger', 'Dome', 'Hoyle Pond', 'Young Davidson',
       'Canadian Malartic', 'Roberto', 'James Bay', 'Excelsior',
       'Royal Hill - Rosebel', 'Pay Caro - Rosebel', 'Salsigne']
localities_2 = ['Essakane', 'Navachab', 'New Consort', 'Hira Buddini', 'Uti',
       'Big Bell', 'St. Ives', 'Lincoln Hill', 'Tehery Wager', 'LaRonde']

# select a sub-set of the original dataset with localities from the lists
pc_aggregated_1 = pc_aggregated[pc_aggregated['LOCALITY'].isin(localities_1)]
pc_aggregated_2 = pc_aggregated[pc_aggregated['LOCALITY'].isin(localities_2)]

fig = px.scatter(pc_aggregated_1, x='PC1', y='PC2', color='LOCALITY', facet_col='LOCALITY', facet_col_wrap=4)
fig.show()
# alternatively, plot density contours: this would work well for larger datasets
# fig = px.density_contour(pc_aggregated_1, x='PC1', y='PC2', color='LOCALITY', facet_col='LOCALITY', facet_col_wrap=4)
# fig.show()

In [51]:
fig = px.scatter(pc_aggregated_2, x='PC1', y='PC2', color='LOCALITY', facet_col='LOCALITY', facet_col_wrap=4)
fig.show()

In [52]:
fig = px.scatter(pc_aggregated, x='PC1', y='PC2', color='LOCALITY', facet_col='ZONE', facet_col_wrap=4)
fig.show()

In [53]:
fig = px.density_contour(pc_aggregated, x='PC1', y='PC2', color='LOCALITY', facet_col='ZONE', facet_col_wrap=4)
fig.show()
# mainly overlapping contours, or a small difference of core vs rim samples in terms of higher PC2 for core samples? 
# by clicking on each item of the legend, it is possible to show / hide contours from different localities